## Instalacion e importacion de dependencias 

In [2]:
!pip install mediapipe opencv-python

     --------------------------------------- 50.3/50.3 MB 10.6 MB/s eta 0:00:00
     --------------------------------------- 38.1/38.1 MB 11.5 MB/s eta 0:00:00
  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl (904 kB)
     ---------------------------------------- 199.7/199.7 KB ? eta 0:00:00
     --------------------------------------- 44.8/44.8 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.3
    Uninstalling protobuf-4.23.3:
      Successfully uninstalled protobuf-4.23.3


You should consider upgrading via the 'C:\Users\lucas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import mediapipe as mp 
import numpy as np

mp_drawing = mp.solutions.drawing_utils

mp_pose = mp.solutions.pose  #pose estimation model 

In [ ]:
# VIDEO FEED 
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("Mediapipe Feed", frame)

    if cv2.waitKey(10) & 0XFF == ord("q"):
        break

cap.release()

cv2.destroyAllWindows()

## Make Detections

In [8]:
cap = cv2.VideoCapture(0)

#set up mediapipe instance 
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection 
        results = pose.process(image)

        # recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Render detections 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0XFF == ord("q"):
            break

cap.release()

cv2.destroyAllWindows()

 ## Determining Joints 

<img src="imgs/joints.png">

In [ ]:
cap = cv2.VideoCapture(0)

#set up mediapipe instance 
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection 
        results = pose.process(image)

        # recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks 
        try:
            landmarks = results.pose_landmarks.landmark 

            print(landmarks)
        except:
            pass

        #Render detections 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0XFF == ord("q"):
            break

cap.release()

cv2.destroyAllWindows()

In [10]:
len(landmarks)

33

In [ ]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

In [ ]:
print(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])
print("--------------------------------------------")
print(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
print("--------------------------------------------")
print(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])


## Calculate Angles 

In [2]:
def calculate_angle(a, b, c):

    a = np.array(a) # first

    b = np.array(b) # mid

    c = np.array(c) # end 

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 100.0:
        angle = 360 - angle 

    return angle 

In [5]:
shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]

elbow = [
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]

wrist = [
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]

In [16]:
calculate_angle(shoulder, elbow, wrist)

236.9932193630114

In [ ]:
cap = cv2.VideoCapture(0)

#set up mediapipe instance 
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection 
        results = pose.process(image)

        # recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks 
        try:
            landmarks = results.pose_landmarks.landmark 

            # Get Coordinates 
            shoulder = [
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]

            elbow = [
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]

            wrist = [
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]
            
            #calculate angle 
            angle = calculate_angle(shoulder, elbow, wrist)

            #vizualize angle
            cv2.putText(image,
                        str(angle),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
        except:
            pass

        #Render detections 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0XFF == ord("q"):
            break

cap.release()

cv2.destroyAllWindows()

## Curl Counter

In [12]:
cap = cv2.VideoCapture(0)

# curl counter variables
counter = 0

stage = None 

#set up mediapipe instance 
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection 
        results = pose.process(image)

        # recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks 
        try:
            landmarks = results.pose_landmarks.landmark 

            # Get Coordinates 
            shoulder = [
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]

            elbow = [
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]

            wrist = [
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]
            
            # calculate angle 
            angle = calculate_angle(shoulder, elbow, wrist)

            # vizualize angle
            cv2.putText(image,
                        str(angle),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            
            # curl counter logic
            if angle > 160: 
                stage = "down"

            if angle < 30 and stage == "down": 
                stage = "up"
                counter += 1
                print(counter)

        except:
            pass

        # Render Curl Counter
        # set up status box 
        cv2.rectangle(image, (0,0), (275,73), (245, 117, 16), -1)

        #rep data 
        cv2.putText(image, "REPS", (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA ) 
        
        cv2.putText(image, str(counter), (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (155, 255, 255), 2, cv2.LINE_AA ) 
        
        # Stage data
        cv2.putText(image, 'STAGE', (90,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, stage, 
                    (90,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        #Render detections 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 ) 

        

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0XFF == ord("q"):
            break

cap.release()

cv2.destroyAllWindows()

1
2
3
4
5
